In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.optimizers import Adam
import os
np.random.seed(2017)

Using TensorFlow backend.
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train_data = pd.read_csv('../../data/spooky-author/download/train.csv')
mapper = {'EAP':0, 'HPL':1, 'MWS':2}
train_data['author'] = train_data['author'].map(lambda x : mapper[x])

In [3]:
texts = train_data['text'].values.tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=90)
labels = to_categorical(train_data['author'].values.tolist())
print('data:', data.shape)
print('label:', labels.shape)

data: (19579, 90)
label: (19579, 3)


In [4]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.20 * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [5]:
embeddings_index = {}

f = open(os.path.join('../../src/pip/glove/glove.6B.50d.txt'))

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

del embeddings_index

embedding_layer = Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], input_length=90, trainable=True)

In [6]:
model = Sequential()
model.add(embedding_layer)
model.add(GlobalAveragePooling1D())
model.add(Dense(3, activation='softmax'))
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [7]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 20
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.fit(x_train, y_train, **params, )

Train on 15664 samples, validate on 3915 samples
Epoch 1/20
15664/15664 [==============================] - 13s 840us/step - loss: 1.0794 - acc: 0.4054 - val_loss: 1.0642 - val_acc: 0.4286
Epoch 2/20
15664/15664 [==============================] - 13s 836us/step - loss: 1.0557 - acc: 0.4348 - val_loss: 1.0441 - val_acc: 0.4557
Epoch 3/20
15664/15664 [==============================] - 13s 831us/step - loss: 1.0336 - acc: 0.4725 - val_loss: 1.0228 - val_acc: 0.4797
Epoch 4/20
15664/15664 [==============================] - 13s 832us/step - loss: 1.0092 - acc: 0.5048 - val_loss: 0.9995 - val_acc: 0.5086
Epoch 5/20
15664/15664 [==============================] - 13s 830us/step - loss: 0.9821 - acc: 0.5366 - val_loss: 0.9741 - val_acc: 0.5499
Epoch 6/20
15664/15664 [==============================] - 13s 833us/step - loss: 0.9524 - acc: 0.5739 - val_loss: 0.9470 - val_acc: 0.5778
Epoch 7/20
15664/15664 [==============================] - 13s 831us/step - loss: 0.9210 - acc: 0.6069 - val_loss: 0.9

In [ ]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 30
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0001
model.fit(x_train, y_train, **params)

Train on 15664 samples, validate on 3915 samples
Epoch 1/30
11528/15664 [=====================>........] - ETA: 3s - loss: 0.5510 - acc: 0.8149